Link to more in-depth descriptions here: https://blackrockneurotech.com/research/wp-content/ifu/LB-0023-7.00_NEV_File_Format.pdf

**Nev obj structure**

It has three main attributes/functions: basic_header, getdata(), extended_headers. The documentation mentions others, such as processroicommments(), but the files we have do not have those.

nevobj.basic_header returns a dictionary with the following keys/values:
- **key**: 'FileTypeID', **value**: str (e.g. 'NEURALEV')
- **key**: 'FileSpec', **value**: str with float (e.g. '2.3')
- **key**: 'AddFlags', **value**: int (likely bool 1/0)
- **key**: 'BytesInHeader', **value**: int
- **key**: 'BytesInDataPackets', **value**: int
- **key**: 'TimeStampResolution', **value**: int
- **key**: 'SampleTimeResolution', **value**: int
- **key**: 'TimeOrigin', **value**: datetime.datetime
- **key**: 'CreatingApplication', **value**: str (e.g. 'File Dialog v7.0.4')
- **key**: 'Comment', **value**: str
- **key**: 'NumExtendedHeaders', **value**: int

nevobj.getdata() takes a long time and returns a dictionary with the following structure:
- **key**: spike_events, **value**: dict
    - **key**: TimeStamps, **value**: list
        - A list of times (integers) in ascending order at which spikes occur (**note**: this is NOT the same as the total duration of the session)
        - The length of this list should, in theory, equal the number of spikes (aka threshold crossings)
    - **key**: Unit, **value**: list
        - A list whose length is equal to Timestamps. In all files I've opened, this list contains only 0s
    - **key**: Channel, **value**: list
        - A list that contains the channel number that corresponds to the spike event time in TimeStamps
        - If the first entry of Timestamps is 30 and the first entry of Channel is 2, that means that a spike occurred in channel 2 at time 30
    - **key**: Waveforms, ****value****: array
        - Array shape: num timestamps x num channels
        - The columns of this array contain the activity of the corresponding channel
- **key**: digital_events, **value**: dict (**note: not all files have digital_events - FR and Cage usually do not**)
    - **key**: Timestamps, **value**: list
        - Not the same timestamps as spike_events. Different lengths and values.
    - **key**: InsertionReason, **value**: list
        - A list whose length is equal to Timestamps. In all files I've opened, this list contains only 1s
    - **key**: UnparsedData, **value**: list
        - A list with integers that encode various task- and trial-related information. Details can be found here:  https://github.com/limblab/Behavior/blob/master/src/target/words.h
        
nevobj.extended_headers returns a list of dicts; the number of dicts equals 'NumExtendedHeaders' in nevobj.basic_headers. 3 dicts in a row correspond to one electrode and contain the following info:
- dict1
    - **key**: 'PacketID':, **value**: str (e.g. 'NEUEVWAV')
    - **key**: 'ElectrodeID':, **value**:  int
    - **key**: 'PhysicalConnector':, **value**: int
    - **key**: 'ConnectorPin':, **value**:  int,
    - **key**: 'DigitizationFactor':, **value**:  int,
    - **key**: 'EnergyThreshold':, **value**:  int,
    - **key**: 'HighThreshold':, **value**:  int,
    - **key**: 'LowThreshold':, **value**:  int,
    - **key**: 'NumSortedUnits':, **value**:  int,
    - **key**: 'BytesPerWaveform':, **value**:  int,
    - **key**: 'SpikeWidthSamples':, **value**:  int,
    - **key**: 'EmptyBytes':, **value**: bytes
- dict2: electrode number
    - **key**: PacketID:, **value**: str 'NEUEVLBL'
    - **key**: ElectrodeID:, **value**: int (e.g. 1, should correspond to dict1)
    - **key**: Label:, **value**: str (the actual electrode number - e.g. 'elec78')
    - **key**: EmptyBytes:, **value**:  bytes (e.g. b'\x00\x00\x00\x00\x00\x00')
- dict3: filter information (type, frequency)
    - **key**: PacketID, **value**: str (e.g. 'NEUEVFLT')
    - **key**: ElectrodeID, **value**: int (e.g. 1, should corresond to dicts 1,2)
    - **key**: HighFreqCorner, **value**: str with float (e.g. '250.0 Hz')
    - **key**: HighFreqOrder, **value**: int (e.g. 4),
    - **key**: HighFreqType, **value**: str (e.g. 'butterworth')
    - **key**: LowFreqCorner, **value**: str with float (e.g. '7500.0 Hz')
    - **key**: LowFreqOrder, **value**: int (e.g. 3)
    - **key**: LowFreqType, **value**: str (e.g. 'butterworth')
    - **key**: EmptyBytes, **value**: bytes
        
**Nsx obj structure**

nsxobj.basic_header produces a dictionary with the following structure: 
- **key**: 'FileTypeID', **value**: str (e.g.  'NEURALCD')
- **key**: 'FileSpec', **value**: str with float (e.g.  '2.3')
- **key**: 'BytesInHeader', **value**: int (e.g.  8762)
- **key**: 'Label', **value**: str representing a rate (e.g.  '2 kS/s')
- **key**: 'Comment', **value**: str 
- **key**: 'Period', **value**: int (e.g.  15)
- **key**: 'TimeStampResolution', **value**: int (e.g.  30000)
- **key**: 'TimeOrigin', **value**:(e.g.  datetime.datetime(2023, 2, 14, 21, 41, 36, 14000))
- **key**: 'ChannelCount', **value**: int (e.g.  128)

nsxobj.getdata() takes a few seconds a dict with the following structure:
- **key**: elec_ids, **value**: list
    - List of electrode id's
- **key**: start_time_s, **value**: float
    - Usually 0.
- **key**: data_time_s, **value**: str
    - In all of the files I've opened, this string has been 'all'.
- **key**: downsample, **value**: int
    - Likely boolean - 1s or 0s. In all the files I've opened, it's been 1.
- **key**: data, **value**: list
    - A list containing a numpy array. This contains emg, force, and other data.
- **key**: data_headers, **value**: list whose only element is a dict
    - **key**: Timestamp, **value**: int (always 0 from what I've seen)
    - **key**: NumDataPoints, **value**: int (with number of total data points - should equal time length of file times sampling rate)
- **key**: ExtendedHeaderIndices, **value**: list
    - A list containing the unique electrode ids - i.e. the length of this list equals the number of electrodes. Values usually between 1 - num electrodes.

nsxobj.extended_headers produces a list of dicts, one for each electrode. The dicts contain the following info:
- **key**: 'Type', **value**: str (e.g. 'CC')
- **key**: 'ElectrodeID', **value**: int (e.g. 1)
- **key**: 'ElectrodeLabel', **value**: str (e.g. 'elec109')
- **key**: 'PhysicalConnector', **value**: int (e.g. 1)
- **key**: 'ConnectorPin', **value**: int (e.g. 1)
- **key**: 'MinDigitalValue', **value**: int (e.g. -32764)
- **key**: 'MaxDigitalValue', **value**: int (e.g. 32764)
- **key**: 'MinAnalogValue', **value**: int (e.g. -8191)
- **key**: 'MaxAnalogValue', **value**: int (e.g. 8191)
- **key**: 'Units', **value**: str (e.g. 'uV')
- **key**: 'HighFreqCorner', **value**: str with float in hz (e.g. '0.3 Hz')
- **key**: 'HighFreqOrder', **value**: int (e.g. 1)
- **key**: 'HighFreqType', **value**: str (e.g. 'butterworth')
- **key**: 'LowFreqCorner', **value**: str with float in hz (e.g. '250.0 Hz')
- **key**: 'LowFreqOrder', **value**: int (e.g. 4)
- **key**: 'LowFreqType', **value**: str (e.g. 'butterworth')

Force Files

- sessions_key - from sessions table
- paper_key - ignore for now
- filename
- file_id - auto
- **rec_system** - 'Cerebus'
- sampling_rate - based on nsx file type
- **force_quality** - leave for now
- **force_notes** - leave for now
- **force_labels - sometimes in nsx extended headers, but not always - seems to be only sometimes for FR files - e.g. NsxFileObj.extended_headers[hdr_idx]['ElectrodeLabel'] returns Force_x** - FR should not have forces, but...?

In [3]:
# Import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
from os import path, system
import sys
from sys import platform
import glob
# from PyQt5.QtWidgets import QFileDialog

# brpylib is the module that contains functions/classes that allow us to open and extract data from .nev and .nsx files
sys.path.insert(0, r'C:\Users\aqy3283\Desktop\proc-kevin')
from Blackrock_Python_Utilities import brpylib

In [4]:
sampling_rate_dict = {'ns1': 500, 'ns2': 1000, 'ns3': 2000, 'ns4': 10000, 'ns5': 30000, 'ns6': 30000}

In [5]:
sampling_rate_dict

{'ns1': 500,
 'ns2': 1000,
 'ns3': 2000,
 'ns4': 10000,
 'ns5': 30000,
 'ns6': 30000}

In [6]:
cerebus_data_dict = {}
base_dir = 'Z:\data'
for monkey in sorted(os.listdir(base_dir)):
#     if monkey not in ['.DS_Store','archive','Backed_up_data', 'Behavior','chewie-delete','CompiledCOFiles','DeepLabCutVids','DLC_models','DPZ','FSMIT_DataRestore_03172021', 'Han_13B1_target','IMU','Jarvis','Jango_redo','Jango_target_redo','LoadCell','Mihili_12A3_target','OldCerebusTest','Rats','Rats_target','Test data','Thumbs.db']:
    if (monkey == 'Pancake_20K3') or (monkey == 'Pop_18E3'):
        print(monkey)
        cerebus_data_dict[monkey] = {}
        monkey_path = os.path.join(base_dir, monkey)
        x = [i for i in os.listdir(monkey_path) if 'cerebus' in i.lower()]
        if len(x) != 0:
            cerebus_path = os.path.join(monkey_path, x[0])
        else:
            cerebus_path = monkey_path
        print(cerebus_path)
        nev_list = glob.glob(f"{cerebus_path}\*\*.nev")
        nsx_list = glob.glob(f"{cerebus_path}\*\*.ns*")
        ccf_list = glob.glob(f"{cerebus_path}\*\*.ccf")
        print(len(nev_list), len(nsx_list), len(ccf_list))
        cerebus_data_dict[monkey]['nev_list'] = nev_list
        cerebus_data_dict[monkey]['nsx_list'] = nsx_list
        cerebus_data_dict[monkey]['ccf_list'] = ccf_list

Pancake_20K3
Z:\data\Pancake_20K3\Cerebus_data
116 88 90
Pop_18E3
Z:\data\Pop_18E3\CerebusData
768 858 607


# Remember to change code in brpylib.py to allow for large files > 1GB!!

In [7]:
force_dict = {'filename': [], 'rec_system': [], 'sampling_rate': [], 'force_labels': []}

for monkey in cerebus_data_dict.keys():
    for file in cerebus_data_dict[monkey]['nsx_list']:
        filename = file.split('\\')[-1]
        if (('FR' not in filename) and ('free' not in filename.lower()) and ('cage' not in filename.lower())):
            nsxobj = brpylib.NsxFile(file)
            output_nsx = nsxobj.getdata()
            if output_nsx == 0:
                continue
            force_labels_lst = []
            for plot_chan in output_nsx['elec_ids']:
                ch_idx  = output_nsx['elec_ids'].index(plot_chan)
                hdr_idx = output_nsx['ExtendedHeaderIndices'][ch_idx]
                label = nsxobj.extended_headers[hdr_idx]['ElectrodeLabel']
                force_labels_lst.append(label)
                force_labels_lst.append(',')
            full_labels_str = ''.join(force_labels_lst)[:-1]
            force_dict['force_labels'].append(full_labels_str)    
            force_dict['filename'].append(filename)
            force_dict['rec_system'].append('Cerebus')
            force_dict['sampling_rate'].append(sampling_rate_dict[filename[-3:]])


Z:\data\Pancake_20K3\Cerebus_data\20210821\20210821_Pancake_PG_002.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210823\20210823_Pancake20K2_PG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210831\20210831_Pancake_KG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210911\20210911_Pancake_PG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210913\20210913_Pancake_PG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210917\20210917_Pancake_PG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210917\20210917_Pancake_PG_002.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210920\20210920_Pancake_KG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20210920\20210920_Pancake_KG_002.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20220121\20220121_Pancake_PG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20220124\20220124_Pancake_PG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20220125\Pancake_20220125_PG_001.ns3 opened

Z:\data\Pancake_20K3\Cerebus_data\20220729\2022


Z:\data\Pop_18E3\CerebusData\20190620\20190620_Pop_horiz_WM_005.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190703\20190703_Pop_horizWM_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190703\20190703_Pop_horizWM_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190708\20190708_Pop_horizWM_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190708\20190708_Pop_horizWM_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190708\20190708_Pop_horizWM_003.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190710\20190710_Pop_horizWM_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190710\20190710_Pop_horizWM_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190729\20190729_Pop_wm001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190730\20190730_Pop_wm_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190730\20190730_Pop_wm_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190807\20190807_Pop_wm_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190807\20190807_Pop_wm_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20190808\20190808_Po


Z:\data\Pop_18E3\CerebusData\20200310\Pop_20200310_WM_CO_001.ns6 opened

Output data requested is larger than 1 GB, skipping

Z:\data\Pop_18E3\CerebusData\20200310\Pop_20200310_WM_CO_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20200310\Pop_20200310_WM_CO_002.ns6 opened

Output data requested is larger than 1 GB, skipping

Z:\data\Pop_18E3\CerebusData\20200310\Pop_20200310_WM_CO_003.ns3 opened

Z:\data\Pop_18E3\CerebusData\20200310\Pop_20200310_WM_CO_003.ns6 opened

Output data requested is larger than 1 GB, skipping

Z:\data\Pop_18E3\CerebusData\20200311\Pop_20200311_WM_CO_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20200311\Pop_20200311_WM_CO_001.ns6 opened

Output data requested is larger than 1 GB, skipping

Z:\data\Pop_18E3\CerebusData\20200316\Pop_20200316_WM_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20200316\Pop_20200316_WM_002.ns6 opened

Output data requested is larger than 1 GB, skipping

Z:\data\Pop_18E3\CerebusData\20200316\Pop_20200316_WM_003.ns3 opened

Z:\data\Pop_1


Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan21stim_25uA_031.ns5 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan21stim_25uA_031_spikesExtracted.ns5 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan27and16stim_25uA_016.ns3 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan27and16stim_25uA_016.ns5 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan27and16stim_25uA_016_spikesExtracted.ns5 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan27stim_25uA_014.ns3 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan27stim_25uA_014.ns5 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan27stim_25uA_014_spikesExtracted.ns5 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EMG_chan27stim_50uA_015.ns3 opened

Z:\data\Pop_18E3\CerebusData\20201023\Pop_20201023_LeftM1_WM_EM


Z:\data\Pop_18E3\CerebusData\20210602\Pop_20210602_PG_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210602\Pop_20210602_PG_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210602\Pop_20210602_PG_003.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210609\Pop_20210609_PG_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210609\Pop_20210609_PG_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210610\20210610_Pop_WS_Post_Caff_01.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210610\20210610_Pop_WS_Post_Caff_02.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210610\20210610_Pop_WS_Post_Caff_03.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210610\20210610_Pop_WS_Pre_Caff_01.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210611\charging_circuit_noise.ns6 opened

Z:\data\Pop_18E3\CerebusData\20210611\Pop_20210611_001.ns6 opened

Z:\data\Pop_18E3\CerebusData\20210611\Pop_20210611_002.ns6 opened

Output data requested is larger than 1 GB, skipping

Z:\data\Pop_18E3\CerebusData\20210611\Pop_20210611_PG_001.ns3 opened

Z:\d


Z:\data\Pop_18E3\CerebusData\20210902\20210902_Pop_PG_Post_Lex_01.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210902\20210902_Pop_PG_Pre_Lex_01.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210902\20210902_Pop_WS_Post_Lex_01.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210902\20210902_Pop_WS_Pre_Lex_01.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210908\Pop_20210908_MG_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210908\Pop_20210908_MG_002.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210908\Pop_20210908_Treats_001.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210917\20210917_Pop_PG_Post_Lex_04.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210917\20210917_Pop_PG_Pre_Lex_01.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210917\20210917_Pop_WS_Post_Lex_03.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210917\20210917_Pop_WS_Pre_Lex_02.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210922\20210922_Pop_PG_Post_Con_04.ns3 opened

Z:\data\Pop_18E3\CerebusData\20210922\20210922_Pop_PG_Pre_Con_01.ns3 opened

Z:\data\Po

In [8]:
for key in force_dict.keys():
    print(len(force_dict[key]))

483
483
483
483


In [9]:
df = pd.DataFrame(force_dict)

In [10]:
df.force_labels.unique()

array(['Fx,Fy',
       'elec109,elec110,elec97,elec85,elec83,elec73,elec84,elec61,elec72,elec49,elec60,elec48,elec59,elec37,elec70,elec36,elec71,elec24,elec58,elec35,elec45,elec23,elec47,elec22,elec46,elec21,elec34,elec20,elec33,elec19,elec32,elec18,elec108,elec96,elec107,elec95,elec106,elec94,elec105,elec93,elec104,elec81,elec92,elec80,elec103,elec68,elec91,elec82,elec90,elec69,elec79,elec56,elec78,elec57,elec66,elec67,elec54,elec55,elec42,elec43,elec30,elec44,elec29,elec31,elec128,elec120,elec127,elec119,elec126,elec118,elec125,elec117,elec124,elec116,elec123,elec115,elec122,elec114,elec121,elec113,elec112,elec102,elec100,elec101,elec88,elec89,elec77,elec76,elec65,elec64,elec53,elec52,elec41,elec40,elec17,elec28',
       'EMG_PT,EMG_APB,EMG_FPB,EMG_Lum,EMG_ECU,EMG_ECU/ECR,EMG_ECR,EMG_EDC1,EMG_EDC2,EMG_Tri,EMG_Bi,video_sync',
       'elec109,elec110,elec97,elec85,elec83,elec73,elec84,elec61,elec72,elec49,elec60,elec48,elec59,elec37,elec70,elec36,elec71,elec24,elec58,elec35,elec45,elec

In [36]:
df[df['force_labels'].str.contains('Fx|Fy|Force_x|Force_y')]

,filename,rec_system,sampling_rate,force_labels
0,20210821_Pancake_PG_002.ns3,Cerebus,2000,"Fx,Fy"
1,20210823_Pancake20K2_PG_001.ns3,Cerebus,2000,"Fx,Fy"
2,20210831_Pancake_KG_001.ns3,Cerebus,2000,"Fx,Fy"
3,20210911_Pancake_PG_001.ns3,Cerebus,2000,"Fx,Fy"
4,20210913_Pancake_PG_001.ns3,Cerebus,2000,"Fx,Fy"
...,...,...,...,...
478,Pop_20220328_PG_002.ns3,Cerebus,2000,"ainp13,Fx,Fy"
479,20220512_Pop_RT_IsoBox_001.ns2,Cerebus,1000,"TP,Fx,Fy"
480,20220512_Pop_RT_IsoBox_002.ns2,Cerebus,1000,"TP,Fx,Fy"
481,20220623_Pop_iso_001.ns2,Cerebus,1000,"Touchpad,Fx,Fy"


In [37]:
df_filtered = df[df['force_labels'].str.contains('Fx|Fy|Force_x|Force_y')]

In [38]:
df_filtered['force_labels'] = 'Fx,Fy'

C:\Users\aqy3283\AppData\Local\Temp\ipykernel_65556\530678393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['force_labels'] = 'Fx,Fy'


In [42]:
df_filtered.reset_index().drop('index', axis = 1)

,filename,rec_system,sampling_rate,force_labels
0,20210821_Pancake_PG_002.ns3,Cerebus,2000,"Fx,Fy"
1,20210823_Pancake20K2_PG_001.ns3,Cerebus,2000,"Fx,Fy"
2,20210831_Pancake_KG_001.ns3,Cerebus,2000,"Fx,Fy"
3,20210911_Pancake_PG_001.ns3,Cerebus,2000,"Fx,Fy"
4,20210913_Pancake_PG_001.ns3,Cerebus,2000,"Fx,Fy"
...,...,...,...,...
233,Pop_20220328_PG_002.ns3,Cerebus,2000,"Fx,Fy"
234,20220512_Pop_RT_IsoBox_001.ns2,Cerebus,1000,"Fx,Fy"
235,20220512_Pop_RT_IsoBox_002.ns2,Cerebus,1000,"Fx,Fy"
236,20220623_Pop_iso_001.ns2,Cerebus,1000,"Fx,Fy"


In [19]:
df_check_emgs = df[df['force_labels'].str.contains('elec', case = False)]

In [20]:
df_check_emgs

,filename,rec_system,sampling_rate,force_labels
21,20220916_Pancake_PG_Pre_Cyp_02.ns6,Cerebus,30000,"elec109,elec110,elec97,elec85,elec83,elec73,el..."
37,20230214_Pancake_WM_001.ns3,Cerebus,2000,"elec109,elec110,elec97,elec85,elec83,elec73,el..."
38,20230214_Pancake_WM_002.ns3,Cerebus,2000,"elec109,elec110,elec97,elec85,elec83,elec73,el..."
39,20230214_Pancake_WM_003.ns3,Cerebus,2000,"elec109,elec110,elec97,elec85,elec83,elec73,el..."
40,20230216_Pancake_WM_001.ns3,Cerebus,2000,"elec109,elec110,elec97,elec85,elec83,elec73,el..."
...,...,...,...,...
367,Pop_20210618_BatteryTest_Transmitter2_5V.ns6,Cerebus,30000,"elec36,elec35,elec34,elec9"
368,Pop_20210618_BatteryTest_Transmitter2_6V.ns6,Cerebus,30000,"elec36,elec35,elec34,elec9"
380,20210623_SteelCable_016154_6Volts001.ns6,Cerebus,30000,"elec36,elec35,elec34,elec9"
382,20210623_SteelCable_016154_6Volts_PowerSupplyO...,Cerebus,30000,"elec36,elec35,elec34,elec9"


In [23]:
file = cerebus_data_dict['Pancake_20K3']['nsx_list'][0]
nsxobj = brpylib.NsxFile(file)
output_nsx = nsxobj.getdata()


Z:\data\Pancake_20K3\Cerebus_data\20210821\20210821_Pancake_PG_002.ns3 opened


In [25]:
import matplotlib.pyplot as plt
for plot_chan in output_nsx['elec_ids']:

    ch_idx  = output_nsx['elec_ids'].index(plot_chan)
    print('Channel: '+str(plot_chan))
    hdr_idx = output_nsx['ExtendedHeaderIndices'][ch_idx]
    # Bug in the original code - changed np.arange(output_nsx['data'].shape[1] to np.arange(output_nsx['data'][0].shape[0]
    t       = output_nsx['start_time_s'] + np.arange(output_nsx['data'][0].shape[0]) / 2000#output_nsx['samp_per_s']

    plt.plot(t, output_nsx['data'][0][:,ch_idx])
#     plt.axis([t[0], t[-1], np.min(output_nsx['data'][0][:, ch_idx]), np.max(output_nsx['data'][0][:, ch_idx])])
    plt.locator_params(axis='y', nbins=20)
    plt.xlabel('Time (s)')
    plt.ylabel("Output (" + NsxFileObj.extended_headers[hdr_idx]['Units'] + ")")
    plt.title(NsxFileObj.extended_headers[hdr_idx]['ElectrodeLabel'])
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'